In [1]:
import pandas as pd
import requests

In [2]:
import random
import numpy as np
import pandas as pd
from scipy.stats import norm

In [3]:
league_id = 384052481958785024
base_url = 'https://api.sleeper.app/v1/league/' + str(league_id) + '/'

## Get Sleeper User-Details

In [4]:
teams = requests.get(base_url + 'users')

In [5]:
team_lst = teams.json()

In [6]:
team_tups = {}

In [7]:
for val in team_lst:
    team_tups[val['display_name']] = val['user_id']

In [8]:
team_tups

{'msubbaiah': '307654863477538816',
 'dhaksubb': '307663062192619520',
 'bhargavesh': '307664367791390720',
 'rbachal': '307942185758621696',
 'GADI': '307946516406890496',
 'SAADMONIUSREX': '309778749241454592',
 'ssudhish10': '309848813416226816',
 'cvalluru': '310592766369361920',
 'mmj762': '332232044425797632',
 'arvindsivak': '463448194244669440'}

In [9]:
df = pd.DataFrame.from_dict(team_tups,orient='index').reset_index()
df.columns = ['user_name','user_id']
df['roster_id'] = 0

In [10]:
df.head()

,user_name,user_id,roster_id
0,msubbaiah,307654863477538816,0
1,dhaksubb,307663062192619520,0
2,bhargavesh,307664367791390720,0
3,rbachal,307942185758621696,0
4,GADI,307946516406890496,0


Add in roster information as well to the lookup dataframe. 

In [11]:
rosters = requests.get(base_url + 'rosters')

In [12]:
rj = rosters.json()

In [13]:
for val in rj:
    df.loc[df['user_id'] == val['owner_id'],'roster_id'] = val['roster_id']

In [14]:
user_roster_map = df.set_index('roster_id').T.to_dict('records')[0]

In [15]:
user_roster_map

{1: 'msubbaiah',
 2: 'dhaksubb',
 3: 'bhargavesh',
 4: 'rbachal',
 5: 'GADI',
 6: 'SAADMONIUSREX',
 8: 'ssudhish10',
 9: 'cvalluru',
 7: 'mmj762',
 10: 'arvindsivak'}

## Get week to week data

In [16]:
def extract_pts(lst,dct=None):
    if(dct is None):
        dct = {}
        for l in lst:
            dct[l['roster_id']] = [l['points']]
    else: 
        for l in lst: 
            dct[l['roster_id']].append(l['points'])
    return(dct)

In [17]:
weeks = range(1,8)

In [18]:
for i in weeks:
    wk = requests.get(base_url + 'matchups/' + str(i))
    wk_vals = wk.json()
    if(i==1):
        res_dct = extract_pts(wk_vals)
    res_dct = extract_pts(wk_vals,res_dct)

In [19]:
res_dct

{1: [108.2699966430664,
  108.2699966430664,
  134.86000061035156,
  113.54000091552734,
  107.69999694824219,
  207.38999938964844,
  147.25,
  101.37000274658203],
 2: [130.6999969482422,
  130.6999969482422,
  95.06999969482422,
  160.02000427246094,
  107.80000305175781,
  115.68000030517578,
  94.69999694824219,
  116.9800033569336],
 3: [128.08999633789062,
  128.08999633789062,
  99.25,
  146.08999633789062,
  81.80000305175781,
  125.2699966430664,
  156.8000030517578,
  83.62000274658203],
 4: [116.5999984741211,
  116.5999984741211,
  104.70999908447266,
  108.44000244140625,
  131.3699951171875,
  146.1199951171875,
  88.47000122070312,
  111.55999755859375],
 5: [100.9800033569336,
  100.9800033569336,
  118.05999755859375,
  119.79000091552734,
  74.44999694824219,
  111.19999694824219,
  133.9600067138672,
  119.80999755859375],
 6: [178.77000427246094,
  178.77000427246094,
  112.04000091552734,
  176.7100067138672,
  157.64999389648438,
  162.00999450683594,
  104.63999

In [20]:
## Create Simulation

In [33]:
def sim(team, team_dict):
    keys = list(team_dict.keys())
    keys.remove(team)
    opponents = np.random.choice(keys,size=11,replace = True)
    wins = 0
    for i, score in enumerate(team_dict[team]):
        if team_dict[opponents[i]][i] < score:
            wins += 1
    return(wins)

In [34]:
def prop_of_reality(team,team_dict):
    results = {}
    for i in range(100000):
        results[i] = sim(team, team_dict)
    SResults = pd.Series(results)
    mu = SResults.mean()
    std = SResults.std()
    print(mu,std)
    p0 = norm.cdf(0, loc=mu, scale = std)
    p1 = norm.cdf(1, loc=mu, scale = std)
    p2 = norm.cdf(2, loc=mu, scale = std)
    p3 = norm.cdf(3, loc=mu, scale = std)
    p4 = norm.cdf(4, loc=mu, scale = std)
    p5 = norm.cdf(5, loc=mu, scale = std)
    p6 = norm.cdf(6, loc=mu, scale=std)
    p7 = norm.cdf(7, loc=mu, scale=std)
    return(pd.Series({"w0":p0, "w1": p1-p0, "w2": p2-p1, \
                     "w3":p3-p2, "w4":p4-p3,'w5':p5-p4,'w6':p6-p5,'w7':1-p7}))

In [35]:
df = pd.DataFrame()

In [38]:

for i in range(0,8):
    print(norm.cdf(i,loc=5.77,scale=0.84))

3.2315965569786447e-12
6.7912157008667475e-09
3.593140181601463e-06
0.00048754164784951625
0.01755259918850321
0.17965866916478546
0.6078844978567616
0.9284420505987429


In [36]:
for team in res_dct.keys():
    tm_name = user_roster_map[team]
    print(tm_name)
    df[tm_name] =prop_of_reality(team,res_dct)

msubbaiah
4.0066 1.1742445352227144
dhaksubb
4.33141 1.0900226114140779
bhargavesh
3.55886 1.1148936856396885
rbachal
3.33087 1.1025276408202231
GADI
2.55775 0.9461045865221379
SAADMONIUSREX
5.77928 0.8429174257884323
mmj762
4.00193 1.0062486772480277
ssudhish10
4.55397 1.1016348646862513
cvalluru
5.00213 1.3326076772746334
arvindsivak
2.8924 1.1119777897720222


In [25]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.index =['0-7','1-6','2-5','3-4','4-3','5-2','6-1','7-0']
df

,msubbaiah,dhaksubb,bhargavesh,rbachal,GADI,SAADMONIUSREX,mmj762,ssudhish10,cvalluru,arvindsivak
0-7,0.000,0.000,0.001,0.001,0.003,0.000,0.000,0.000,0.000,0.005
1-6,0.005,0.001,0.010,0.016,0.047,0.000,0.001,0.001,0.001,0.040
2-5,0.039,0.015,0.071,0.096,0.229,0.000,0.022,0.009,0.011,0.167
3-4,0.152,0.094,0.229,0.268,0.403,0.000,0.137,0.068,0.054,0.328
4-3,0.303,0.269,0.347,0.347,0.255,0.017,0.339,0.227,0.159,0.302
5-2,0.303,0.350,0.247,0.207,0.058,0.160,0.339,0.351,0.274,0.130
6-1,0.153,0.207,0.082,0.057,0.005,0.425,0.137,0.250,0.274,0.026
7-0,0.005,0.007,0.001,0.000,0.000,0.074,0.001,0.013,0.066,0.000


In [28]:
df.sum(axis=0)

msubbaiah       0.961
dhaksubb        0.944
bhargavesh      0.987
rbachal         0.993
GADI            1.000
SAADMONIUSREX   0.677
mmj762          0.978
ssudhish10      0.918
cvalluru        0.840
arvindsivak     0.998
dtype: float64

In [28]:
df.to_csv('Sleeper-2019-Luck-Proj.csv')